Calls C program with parameters that are given in the arrays

In [10]:
from subprocess import call, check_output
import subprocess
import numpy as np

NNodes = [1600,800,400,200]
phifloat = list(np.linspace(0.001, 1, num=500))
MAX_ITER = [100000000]
ITER_step = [500]
filename = "NewData"
redundancy = 1000

Now with multiprocessing:

In [ ]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import itertools


results2 = []
#As above just with a filename dependent on the worker to avoid corruptions. Will create a file for each thread
def RunCProgram(NN, phi, M_Iter, step):
    global filename
    filename = filename + str( multiprocessing.current_process()._identity[0])
    for x in range(redundancy):
        try:
            check_output(["./a.out", str(NN), str(NN/2), str(NN/10), str(phi), str(M_Iter), str(step), filename, str(0), str(0), str(0)])
        except subprocess.CalledProcessError as e:
            raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
    return


if __name__ == '__main__':
    NThreads = multiprocessing.cpu_count() 
    param = []
    param.append(NNodes)
    param.append(phifloat)
    param.append(MAX_ITER)
    param.append(ITER_step)
    InputArray = list(itertools.product(*param))
    comb = len(InputArray)*redundancy
    with Pool(processes=NThreads) as pool:         # start 4 worker processes
        pool.starmap(RunCProgram, InputArray)       # prints "[0, 1, 4,..., 81]"